In [2]:
import pandas as pd
import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Categories

In [3]:
docs_cats = pd.read_csv("../download/documents_categories.csv")
docs_cats.head()

,document_id,category_id,confidence_level
0,1595802,1611,0.92
1,1595802,1610,0.07
2,1524246,1807,0.92
3,1524246,1608,0.07
4,1617787,1807,0.92


In [4]:
docs_cats['value']=docs_cats['confidence_level']
docs_cats['value2']=np.square(docs_cats['value'])
docs_cats.drop(['confidence_level'], axis=1, inplace=True)
docs_cats.head()

,document_id,category_id,value,value2
0,1595802,1611,0.92,0.8464
1,1595802,1610,0.07,0.0049
2,1524246,1807,0.92,0.8464
3,1524246,1608,0.07,0.0049
4,1617787,1807,0.92,0.8464


In [5]:
docs_norms2 = pd.DataFrame({'norm2':docs_cats.groupby(['document_id'])['value2'].sum()}).reset_index()
docs_cats.drop(['value2'], axis=1, inplace=True)
docs_norms2.head()

,document_id,norm2
0,1,0.223334
1,2,0.520000
2,3,0.274941
3,4,0.851300
4,5,0.851300


In [15]:
np.sqrt(docs_norms2[docs_norms2.document_id==1330329])

,document_id,norm2
1291909,1153.39889,0.922659


In [16]:
np.sqrt(docs_norms2[docs_norms2.document_id==1136820])

,document_id,norm2
1107596,1066.217614,0.922659


### Train

In [6]:
page_addoc_dist = pd.read_csv("../generated/final/events_clicks_CTR_addocs_train.csv", usecols=['document_id', 'addoc_id'])
page_addoc_dist.count()

document_id    87141731
addoc_id       87141731
dtype: int64

In [7]:
page_addoc_dist = pd.DataFrame({'size':page_addoc_dist.groupby(['document_id', 'addoc_id']).size()}).reset_index().drop(['size'], axis=1)
page_addoc_dist.count()

document_id    28491064
addoc_id       28491064
dtype: int64

In [8]:
len(page_addoc_dist.document_id.unique()), len(page_addoc_dist.addoc_id.unique())

(756510, 164476)

In [9]:
page_addoc_diffs = page_addoc_dist.merge(docs_cats, left_on='addoc_id', right_on='document_id')
page_addoc_diffs.rename(columns={'document_id_x':'document_id'}, inplace=True)
page_addoc_diffs.drop(['document_id_y'], axis=1, inplace=True)
page_addoc_diffs = page_addoc_diffs.merge(docs_cats, on=['document_id', 'category_id'])
page_addoc_diffs['value_2xy'] = 2*page_addoc_diffs['value_x']*page_addoc_diffs['value_y']
page_addoc_diffs.drop(['category_id', 'value_x', 'value_y'], axis=1, inplace=True)
page_addoc_diffs = pd.DataFrame({'value_2xy':page_addoc_diffs.groupby(['document_id', 'addoc_id'])['value_2xy'].sum()}).reset_index()

page_addoc_diffs.head()

,document_id,addoc_id,value_2xy
0,5,1044364,1.692800
1,12,228823,0.044968
2,12,309413,0.591008
3,12,358632,0.002090
4,12,371327,0.274983


In [10]:
page_addoc_diffs.count()

document_id    9076297
addoc_id       9076297
value_2xy      9076297
dtype: int64

In [11]:
page_addoc_dist = page_addoc_dist.merge(docs_norms2, on=['document_id'])
page_addoc_dist = page_addoc_dist.merge(docs_norms2, left_on=['addoc_id'], right_on=['document_id'])
page_addoc_dist.rename(columns={'document_id_x':'document_id'}, inplace=True)
page_addoc_dist.drop(['document_id_y'], axis=1, inplace=True)
page_addoc_dist = page_addoc_dist.merge(page_addoc_diffs, on=['document_id', 'addoc_id'], how='left').fillna(0)
page_addoc_dist['dist_cats'] = np.sqrt( page_addoc_dist['norm2_x'] + page_addoc_dist['norm2_y'] - page_addoc_dist['value_2xy'] )
page_addoc_dist.drop(['value_2xy'], axis=1, inplace=True)
page_addoc_dist['doc_norm_cats'] = np.sqrt( page_addoc_dist['norm2_x'])
page_addoc_dist.drop(['norm2_x'], axis=1, inplace=True)
page_addoc_dist['ad_norm_cats'] = np.sqrt( page_addoc_dist['norm2_y'])
page_addoc_dist.drop(['norm2_y'], axis=1, inplace=True)

page_addoc_dist.head()

,document_id,addoc_id,dist_cats,doc_norm_cats,ad_norm_cats
0,2,1489042,1.171025,0.721110,0.922659
1,452,1489042,0.999333,0.383884,0.922659
2,515,1489042,0.977675,0.323339,0.922659
3,1608,1489042,0.321107,0.611836,0.922659
4,3260,1489042,1.069416,0.540695,0.922659


In [12]:
page_addoc_dist.to_csv("../generated/final/page_addoc_cats_dist.csv", index=False)

In [13]:
page_addoc_dist.count()

document_id      27700275
addoc_id         27700275
dist_cats        27700275
doc_norm_cats    27700275
ad_norm_cats     27700275
dtype: int64

### Test

In [5]:
page_addoc_dist = pd.read_csv("../generated/final/events_clicks_CTR_addocs_test.csv", usecols=['document_id', 'addoc_id'])
page_addoc_dist.count()

document_id    32225162
addoc_id       32225162
dtype: int64

In [6]:
page_addoc_dist = pd.DataFrame({'size':page_addoc_dist.groupby(['document_id', 'addoc_id']).size()}).reset_index().drop(['size'], axis=1)
page_addoc_dist.count()

document_id    13358295
addoc_id       13358295
dtype: int64

In [7]:
len(page_addoc_dist.document_id.unique()), len(page_addoc_dist.addoc_id.unique())

(502297, 132067)

In [8]:
page_addoc_diffs = page_addoc_dist.merge(docs_cats, left_on='addoc_id', right_on='document_id')
page_addoc_diffs.rename(columns={'document_id_x':'document_id'}, inplace=True)
page_addoc_diffs.drop(['document_id_y'], axis=1, inplace=True)
page_addoc_diffs = page_addoc_diffs.merge(docs_cats, on=['document_id', 'category_id'])
page_addoc_diffs['value_2xy'] = 2*page_addoc_diffs['value_x']*page_addoc_diffs['value_y']
page_addoc_diffs.drop(['category_id', 'value_x', 'value_y'], axis=1, inplace=True)
page_addoc_diffs = pd.DataFrame({'value_2xy':page_addoc_diffs.groupby(['document_id', 'addoc_id'])['value_2xy'].sum()}).reset_index()

page_addoc_diffs.head()

,document_id,addoc_id,value_2xy
0,8,1598289,0.043362
1,11,1342045,1.524648
2,12,228823,0.044968
3,12,371327,0.274983
4,12,476218,0.594429


In [9]:
page_addoc_diffs.count()

document_id    4282293
addoc_id       4282293
value_2xy      4282293
dtype: int64

In [10]:
page_addoc_dist = page_addoc_dist.merge(docs_norms2, on=['document_id'])
page_addoc_dist = page_addoc_dist.merge(docs_norms2, left_on=['addoc_id'], right_on=['document_id'])
page_addoc_dist.rename(columns={'document_id_x':'document_id'}, inplace=True)
page_addoc_dist.drop(['document_id_y'], axis=1, inplace=True)
page_addoc_dist = page_addoc_dist.merge(page_addoc_diffs, on=['document_id', 'addoc_id'], how='left').fillna(0)
page_addoc_dist['dist_cats'] = np.sqrt( page_addoc_dist['norm2_x'] + page_addoc_dist['norm2_y'] - page_addoc_dist['value_2xy'] )
page_addoc_dist.drop(['norm2_x', 'norm2_y', 'value_2xy'], axis=1, inplace=True)

page_addoc_dist.head()

,document_id,addoc_id,dist_cats
0,8,287704,1.304837
1,41490,287704,1.040957
2,57208,287704,1.304837
3,204133,287704,0.969748
4,297088,287704,1.021505


In [11]:
page_addoc_dist.to_csv("../generated/final/page_addoc_cats_dist_test.csv", index=False)

In [12]:
page_addoc_dist.count()

document_id    12997372
addoc_id       12997372
dist_cats      12997372
dtype: int64